In [1]:
import paramiko
from paramiko import client
import re
import time, threading
import docker

In [3]:
class ssh:
    client = None
 
    def __init__(self, address, username, password):
        print("Connecting to server.")
        self.client = client.SSHClient()
        self.client.set_missing_host_key_policy(client.AutoAddPolicy())
        self.client.connect(address, username=username, password=password, look_for_keys=False)
 
    def sendCommand(self, command):
        if(self.client):
            stdin, stdout, stderr = self.client.exec_command(command)
            while not stdout.channel.exit_status_ready():
                # Print data when available
                if stdout.channel.recv_ready():
                    alldata = stdout.channel.recv(4096)
#                     alldata2 = stderr.channel.recv(3096)
                    prevdata = b"1"
#                     prevdata2 = b"1"
#                     alldata2 += prevdata2
            
                    while prevdata:
                        prevdata = stdout.channel.recv(4096)
#                         prevdata2 = stderr.channel.recv(3096)
                        alldata += prevdata
#                         alldata2 += prevdata2
                        
                        x = (str(alldata, "utf8"))
                        print (x)
                        return x
                else:
#                      elif stdout.channel.recv_stderr_ready():
                    alldata = stderr.channel.recv_stderr(4096)
                    prevdata = b"1"
                    
                    while prevdata:
                        prevdata = stdout.channel.recv(4096)
#                         prevdata2 = stderr.channel.recv(3096)
                        alldata += prevdata
#                         alldata2 += prevdata2
                        
                        x = (str(alldata, "utf8"))
                        print (x)
                        return x
                        
        else:
            print("Connection not opened.")
        

In [37]:
connection = ssh("195.167.137.60", "root" , "ubuntu123")

Connecting to server.


In [59]:
connection.sendCommand("lspci | grep -i Sam")

3f:00.0 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824



'3f:00.0 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824\n'

In [60]:
stdout = connection.sendCommand("lspci | grep -i Sam")
y = re.search("^.+:\d+.\d+", str(stdout))
print(y)
pci_id = y.group()
print (pci_id)

3f:00.0 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824

<re.Match object; span=(0, 7), match='3f:00.0'>
3f:00.0


In [61]:
bus = re.search("^..", pci_id)
bus_id = bus.group()
print (bus_id)

dev_func = re.search(":\d+.\d", pci_id)
#print (dev_func)
dev_func_id = dev_func.group()
dev_func_id = dev_func_id.replace(':', '')
print (dev_func_id)

3f
00.0


In [62]:
#connection.sendCommand("su && 1")
vf_command = "echo 32 > /sys/bus/pci/devices/0000\:" + bus_id + "\:" + dev_func_id + "/sriov_numvfs"
print (str(vf_command))
connection.sendCommand(vf_command)
# print (y)

echo 32 > /sys/bus/pci/devices/0000\:3f\:00.0/sriov_numvfs



''

In [63]:
for i in range(1,33):
    connection.sendCommand("nvme delete-ns /dev/nvme0 -n " + str(i))

delete-ns: Success, deleted nsid:1

delete-ns: Success, deleted nsid:2

delete-ns: Success, deleted nsid:3

delete-ns: Success, deleted nsid:4

delete-ns: Success, deleted nsid:5

delete-ns: Success, deleted nsid:6

delete-ns: Success, deleted nsid:7

delete-ns: Success, deleted nsid:8

delete-ns: Success, deleted nsid:9

delete-ns: Success, deleted nsid:10

delete-ns: Success, deleted nsid:11

delete-ns: Success, deleted nsid:12

delete-ns: Success, deleted nsid:13

delete-ns: Success, deleted nsid:14

delete-ns: Success, deleted nsid:15

delete-ns: Success, deleted nsid:16

delete-ns: Success, deleted nsid:17

delete-ns: Success, deleted nsid:18

delete-ns: Success, deleted nsid:19

delete-ns: Success, deleted nsid:20

delete-ns: Success, deleted nsid:21

delete-ns: Success, deleted nsid:22

delete-ns: Success, deleted nsid:23

delete-ns: Success, deleted nsid:24

delete-ns: Success, deleted nsid:25

delete-ns: Success, deleted nsid:26

delete-ns: Success, deleted nsid:27

delete-ns:

In [64]:
for i in range(1,33):
    connection.sendCommand("nvme create-ns /dev/nvme0 -s 461373440 -c 461373440 -f 0 -d 0 -m 0")
    

create-ns: Success, created nsid:1

create-ns: Success, created nsid:2

create-ns: Success, created nsid:3

create-ns: Success, created nsid:4

create-ns: Success, created nsid:5

create-ns: Success, created nsid:6

create-ns: Success, created nsid:7

create-ns: Success, created nsid:8

create-ns: Success, created nsid:9

create-ns: Success, created nsid:10

create-ns: Success, created nsid:11

create-ns: Success, created nsid:12

create-ns: Success, created nsid:13

create-ns: Success, created nsid:14

create-ns: Success, created nsid:15

create-ns: Success, created nsid:16

create-ns: Success, created nsid:17

create-ns: Success, created nsid:18

create-ns: Success, created nsid:19

create-ns: Success, created nsid:20

create-ns: Success, created nsid:21

create-ns: Success, created nsid:22

create-ns: Success, created nsid:23

create-ns: Success, created nsid:24

create-ns: Success, created nsid:25

create-ns: Success, created nsid:26

create-ns: Success, created nsid:27

create-ns:

In [65]:
#connection.sendCommand("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x10008 --cdw11=2")
#connection.sendCommand("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x10108 --cdw11=2")

for i in range(1,21):
    x = connection.sendCommand("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(i) + "0008 --cdw11=2")
    print("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(i) + "0008 --cdw11=2 " + "= " + x)

# list_hexa=[]
# list_hexa.append(0A)
for z in ['0A','0B','0C','0D','0E','0F','1A','1B','1C','1D','1E','1F']:
# for z in list_hexa:
    y = connection.sendCommand("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(z) + "0008 --cdw11=2")
    print("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(z) + "0008 --cdw11=2 " + "= " + y)
    
# connection.sendCommand("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x320008 --cdw11=2")

# import paramiko
# client=paramiko.SSHClient()
# client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# com="ls ~/desktop"
# client.connect('195.167.137.43', username='root', password='1', look_for_keys=False)
# output=""

# # stdin, stdout, stderr = client.exec_command("lspci | grep -i Sam")
# stdin, stdout, stderr = client.exec_command("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x320008 --cdw11=2")
# while not stdout.channel.exit_status_ready():
# #     print(stdout.channel.recv_ready())
# #     print (stdout.channel.recv_stderr_ready())
#     if stdout.channel.recv_stderr_ready():
#         alldata = stderr.channel.recv_stderr(4096)
#         print (alldata)
        
    
    
#     if stderr.channel.recv_ready():       
# #                     alldata2 = stderr.channel.recv(3096)
#         prevdata = b"1"
#         print (prevdata)
    
    
# print (stdout.channel.exit_status_ready())
                # Print data when available
                
#                     prevdata2 = b"1"
#                     alldata2 += prevdata2
                
#                 else: 
#                     while prevdata:
#                         prevdata = stdout.channel.recv(3096)
# #                         prevdata2 = stderr.channel.recv(3096)
#                         alldata += prevdata
# #                         alldata2 += prevdata2
                        
#                         x = (str(alldata, "utf8"))
#                         print (x)
#                         return x

# print ("ssh succuessful. Closing connection")
# stdout=stderr.readlines()
# client.close()
# print ("Connection closed")

# print (stdout)
    


NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x10008 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x20008 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x30008 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x40008 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x50008 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x60008 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x70008 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode

In [66]:
# for Running second command for Alloc VF
for i in range(1,21):
    x = connection.sendCommand("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(i) + "0108 --cdw11=2")
    print("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(i) + "0108 --cdw11=2 " + "= " + x)

# list_hexa=[]
# list_hexa.append(0A)
for z in ['0A','0B','0C','0D','0E','0F','1A','1B','1C','1D','1E','1F']:
# for z in list_hexa:
    y = connection.sendCommand("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(z) + "0108 --cdw11=2")
    print("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(z) + "0108 --cdw11=2 " + "= " + y)

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x10108 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x20108 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x30108 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x40108 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x50108 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x60108 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x70108 --cdw11=2 = NVMe command result:00000002

NVMe command result:00000002

nvme admin-passthru /dev/nvme0 --opcode

In [67]:
list_pci_reset=[]
pci_vf_id = connection.sendCommand("lspci | grep -i Sam")
lspci_list = pci_vf_id.splitlines()
# print (lspci_list)

for x in lspci_list:
    pciid = re.search("^.+:\d+.\d+", str(x))
    pciid = pciid.group()
    list_pci_reset.append(pciid)
# print(list_pci_reset)


for y in list_pci_reset:
    bus = re.search("^..", y)
    bus_id = bus.group()
    dev_func = re.search(":\d+.\d", y)
    dev_func_id = dev_func.group()
    dev_func_id = dev_func_id.replace(':', '')
    vf_command2 = "echo 1 > /sys/bus/pci/devices/0000\:" + bus_id + "\:" + dev_func_id + "/reset"
    connection.sendCommand(vf_command2)
    print ("VF Reset Successfull ==> " + vf_command2)


3f:00.0 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:00.1 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:00.2 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:00.3 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:00.4 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:00.5 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:00.6 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:00.7 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:01.0 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:01.1 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:01.2 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:01.3 Non-Volatile memory controller: Samsung Electronics Co Ltd Device a824
3f:01.4 Non-Volatile memory controller: Samsung Elec

In [68]:
# Cmd to set all VFs online
for i in range(1,21):
    x = connection.sendCommand("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(i) + "0009 --cdw11=0")
    print("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(i) + "0009 --cdw11=0 " + "= " + x)

# list_hexa=[]
# list_hexa.append(0A)
for z in ['0A','0B','0C','0D','0E','0F','1A','1B','1C','1D','1E','1F']:
# for z in list_hexa:
    y = connection.sendCommand("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(z) + "0009 --cdw11=0")
    print("nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x" + str(z) + "0009 --cdw11=0 " + "= " + y)

NVMe command result:00000000

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x10009 --cdw11=0 = NVMe command result:00000000

NVMe command result:00000000

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x20009 --cdw11=0 = NVMe command result:00000000

NVMe command result:00000000

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x30009 --cdw11=0 = NVMe command result:00000000

NVMe command result:00000000

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x40009 --cdw11=0 = NVMe command result:00000000

NVMe command result:00000000

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x50009 --cdw11=0 = NVMe command result:00000000

NVMe command result:00000000

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x60009 --cdw11=0 = NVMe command result:00000000

NVMe command result:00000000

nvme admin-passthru /dev/nvme0 --opcode=0x1C --cdw10=0x70009 --cdw11=0 = NVMe command result:00000000

NVMe command result:00000000

nvme admin-passthru /dev/nvme0 --opcode

In [69]:
for pci in list_pci_reset:
    print ("echo 0000:" + pci + " > /sys/bus/pci/drivers/nvme/bind")
    connection.sendCommand("echo 0000:" + pci + " > /sys/bus/pci/drivers/nvme/bind")
    

echo 0000:3f:00.0 > /sys/bus/pci/drivers/nvme/bind
bash: line 0: echo: write error: No such device

echo 0000:3f:00.1 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:00.2 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:00.3 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:00.4 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:00.5 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:00.6 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:00.7 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:01.0 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:01.1 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:01.2 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:01.3 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:01.4 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:01.5 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:01.6 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:01.7 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:02.0 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:02.1 > /sys/bus/pci/drivers/nvme/bind

echo 0000:3f:02.

In [70]:
for i in range(1,33):
    print("nvme attach-ns /dev/nvme0 -n " + str(i) + " -c " + str(i))
    connection.sendCommand("nvme attach-ns /dev/nvme0 -n " + str(i) + " -c " + str(i))

nvme attach-ns /dev/nvme0 -n 1 -c 1
attach-ns: Success, nsid:1

nvme attach-ns /dev/nvme0 -n 2 -c 2
attach-ns: Success, nsid:2

nvme attach-ns /dev/nvme0 -n 3 -c 3
attach-ns: Success, nsid:3

nvme attach-ns /dev/nvme0 -n 4 -c 4
attach-ns: Success, nsid:4

nvme attach-ns /dev/nvme0 -n 5 -c 5
attach-ns: Success, nsid:5

nvme attach-ns /dev/nvme0 -n 6 -c 6
attach-ns: Success, nsid:6

nvme attach-ns /dev/nvme0 -n 7 -c 7
attach-ns: Success, nsid:7

nvme attach-ns /dev/nvme0 -n 8 -c 8
attach-ns: Success, nsid:8

nvme attach-ns /dev/nvme0 -n 9 -c 9
attach-ns: Success, nsid:9

nvme attach-ns /dev/nvme0 -n 10 -c 10
attach-ns: Success, nsid:10

nvme attach-ns /dev/nvme0 -n 11 -c 11
attach-ns: Success, nsid:11

nvme attach-ns /dev/nvme0 -n 12 -c 12
attach-ns: Success, nsid:12

nvme attach-ns /dev/nvme0 -n 13 -c 13
attach-ns: Success, nsid:13

nvme attach-ns /dev/nvme0 -n 14 -c 14
attach-ns: Success, nsid:14

nvme attach-ns /dev/nvme0 -n 15 -c 15
attach-ns: Success, nsid:15

nvme attach-ns /dev/nv

In [71]:
for i in range(1,33):
    print("nvme reset /dev/nvme" + str(i))
    connection.sendCommand("nvme reset /dev/nvme" + str(i))

nvme reset /dev/nvme1

nvme reset /dev/nvme2

nvme reset /dev/nvme3

nvme reset /dev/nvme4

nvme reset /dev/nvme5

nvme reset /dev/nvme6

nvme reset /dev/nvme7

nvme reset /dev/nvme8

nvme reset /dev/nvme9

nvme reset /dev/nvme10

nvme reset /dev/nvme11

nvme reset /dev/nvme12

nvme reset /dev/nvme13

nvme reset /dev/nvme14

nvme reset /dev/nvme15

nvme reset /dev/nvme16

nvme reset /dev/nvme17

nvme reset /dev/nvme18

nvme reset /dev/nvme19

nvme reset /dev/nvme20

nvme reset /dev/nvme21

nvme reset /dev/nvme22

nvme reset /dev/nvme23

nvme reset /dev/nvme24

nvme reset /dev/nvme25

nvme reset /dev/nvme26

nvme reset /dev/nvme27

nvme reset /dev/nvme28

nvme reset /dev/nvme29

nvme reset /dev/nvme30

nvme reset /dev/nvme31

nvme reset /dev/nvme32



In [4]:
# for i in range(1,33):
#     print("nvme reset /dev/nvme" + str(i))


nvme_namespaces_list=[]
lsblk_nvme = connection.sendCommand("lsblk | grep -i nvme")
lsblk_nvme_list = lsblk_nvme.splitlines()
# print (lsblk_nvme_list)

for x in lsblk_nvme_list:
    namespace = re.search("^nvme\dn\d+", str(x))
    namespace = namespace.group()
    nvme_namespaces_list.append(namespace)
print(nvme_namespaces_list)

nvme0n1  259:3    0    32G  0 disk /mnt/disk1
nvme0n2  259:7    0    32G  0 disk /mnt/disk2
nvme0n3  259:10   0    32G  0 disk /mnt/disk3
nvme0n4  259:12   0    32G  0 disk /mnt/disk4
nvme0n5  259:14   0    32G  0 disk /mnt/disk5
nvme0n6  259:16   0    32G  0 disk /mnt/disk6
nvme0n7  259:18   0    32G  0 disk /mnt/disk7
nvme0n8  259:20   0    32G  0 disk /mnt/disk8
nvme0n9  259:22   0    32G  0 disk /mnt/disk9
nvme0n10 259:24   0    32G  0 disk /mnt/disk10
nvme0n11 259:26   0    32G  0 disk /mnt/disk11
nvme0n12 259:28   0    32G  0 disk /mnt/disk12
nvme0n13 259:30   0    32G  0 disk /mnt/disk13
nvme0n14 259:32   0    32G  0 disk /mnt/disk14
nvme0n15 259:34   0    32G  0 disk /mnt/disk15
nvme0n16 259:36   0    32G  0 disk /mnt/disk16
nvme0n17 259:38   0    32G  0 disk /mnt/disk17
nvme0n18 259:40   0    32G  0 disk /mnt/disk18
nvme0n19 259:42   0    32G  0 disk /mnt/disk19
nvme0n20 259:44   0    32G  0 disk /mnt/disk20
nvme0n21 259:46   0    32G  0 disk /mnt/disk21
nvme0n22 259:48   0   

In [225]:
for i in range(1,33):
    connection.sendCommand("mkdir /mnt/disk" + str(i))
    wait 
    print("Disk" + str(i) + " created successfully")


Disk1 created successfully

Disk2 created successfully

Disk3 created successfully

Disk4 created successfully

Disk5 created successfully

Disk6 created successfully

Disk7 created successfully

Disk8 created successfully

Disk9 created successfully

Disk10 created successfully

Disk11 created successfully

Disk12 created successfully

Disk13 created successfully

Disk14 created successfully

Disk15 created successfully

Disk16 created successfully

Disk17 created successfully

Disk18 created successfully

Disk19 created successfully

Disk20 created successfully

Disk21 created successfully

Disk22 created successfully

Disk23 created successfully

Disk24 created successfully

Disk25 created successfully

Disk26 created successfully

Disk27 created successfully

Disk28 created successfully

Disk29 created successfully

Disk30 created successfully

Disk31 created successfully

Disk32 created successfully


In [229]:
for nvmeid in nvme_namespaces_list:
    connection.sendCommand("mkfs.ext4 /dev/" + nvmeid)
    time.sleep(4)
    print("Successfully created ext4 partition on " + nvmeid)

mke2fs 1.44.1 (24-Mar-2018)
Discarding device blocks:    4096/838860
Successfully created ext4 partition on nvme0n1
mke2fs 1.44.1 (24-Mar-2018)
Discarding device blocks:    4096/838860
Successfully created ext4 partition on nvme0n2
mke2fs 1.44.1 (24-Mar-2018)
Discarding device blocks:    4096/838860
Successfully created ext4 partition on nvme0n3
mke2fs 1.44.1 (24-Mar-2018)
Discarding device blocks:    4096/838860
Successfully created ext4 partition on nvme0n4
mke2fs 1.44.1 (24-Mar-2018)
Discarding device blocks:    4096/838860
Successfully created ext4 partition on nvme0n5
mke2fs 1.44.1 (24-Mar-2018)
Discarding device blocks:    4096/838860
Successfully created ext4 partition on nvme0n6
mke2fs 1.44.1 (24-Mar-2018)
Discarding device blocks:    4096/838860
Successfully created ext4 partition on nvme0n7
mke2fs 1.44.1 (24-Mar-2018)
Discarding device blocks:    4096/838860
Successfully created ext4 partition on nvme0n8
mke2fs 1.44.1 (24-Mar-2018)
Discarding device blocks:    4096/838860
Suc

In [255]:
counter = 0
for nvmeid in nvme_namespaces_list:
    counter = counter + 1
    connection.sendCommand("mount /dev/" + nvmeid + " /mnt/disk" + str(counter))
    print("mount /dev/" + nvmeid + " /mnt/disk" + str(counter) + " successfully !!!")
         
    


mount /dev/nvme0n1 /mnt/disk1 successfully !!!

mount /dev/nvme0n2 /mnt/disk2 successfully !!!

mount /dev/nvme0n3 /mnt/disk3 successfully !!!

mount /dev/nvme0n4 /mnt/disk4 successfully !!!

mount /dev/nvme0n5 /mnt/disk5 successfully !!!

mount /dev/nvme0n6 /mnt/disk6 successfully !!!

mount /dev/nvme0n7 /mnt/disk7 successfully !!!

mount /dev/nvme0n8 /mnt/disk8 successfully !!!

mount /dev/nvme0n9 /mnt/disk9 successfully !!!

mount /dev/nvme0n10 /mnt/disk10 successfully !!!

mount /dev/nvme0n11 /mnt/disk11 successfully !!!

mount /dev/nvme0n12 /mnt/disk12 successfully !!!

mount /dev/nvme0n13 /mnt/disk13 successfully !!!

mount /dev/nvme0n14 /mnt/disk14 successfully !!!

mount /dev/nvme0n15 /mnt/disk15 successfully !!!

mount /dev/nvme0n16 /mnt/disk16 successfully !!!

mount /dev/nvme0n17 /mnt/disk17 successfully !!!

mount /dev/nvme0n18 /mnt/disk18 successfully !!!

mount /dev/nvme0n19 /mnt/disk19 successfully !!!

mount /dev/nvme0n20 /mnt/disk20 successfully !!!

mount /dev/nvme0n

In [14]:
for i in range(1,33):
    
    connection.sendCommand("umount /mnt/disk" + str(i))
    print("umount /mnt/disk" + str(i) + " successfully !!!")

umount: /mnt/disk1: not mounted.

umount /mnt/disk1 successfully !!!

umount /mnt/disk2 successfully !!!

umount /mnt/disk3 successfully !!!

umount /mnt/disk4 successfully !!!

umount /mnt/disk5 successfully !!!

umount /mnt/disk6 successfully !!!

umount /mnt/disk7 successfully !!!

umount /mnt/disk8 successfully !!!

umount /mnt/disk9 successfully !!!

umount /mnt/disk10 successfully !!!

umount /mnt/disk11 successfully !!!

umount /mnt/disk12 successfully !!!

umount /mnt/disk13 successfully !!!

umount /mnt/disk14 successfully !!!

umount /mnt/disk15 successfully !!!

umount /mnt/disk16 successfully !!!

umount /mnt/disk17 successfully !!!

umount /mnt/disk18 successfully !!!

umount /mnt/disk19 successfully !!!

umount /mnt/disk20 successfully !!!

umount /mnt/disk21 successfully !!!

umount /mnt/disk22 successfully !!!

umount /mnt/disk23 successfully !!!

umount /mnt/disk24 successfully !!!

umount /mnt/disk25 successfully !!!

umount /mnt/disk26 successfully !!!

umount /mnt/d

In [15]:
connection.sendCommand("docker rm $(docker ps -aq)")
# connection.sendCommand("mkdir logs")

b2bdd17cf59a
2e7643b01412
9c124ff553a7
beb3ff900509
5e46e48fa372
62700ada4e14
5fa3c994bf84
895703dde361
4e945c1223c5
50f42244bf01
57f0c9ce29b4
0b82a523a46c
0f8a75b14cb0
5f4d7300a2e7
e1eb3ac50c2a
d907274d448e
d8483e94e35b
45845e6c5afb
171bc103e5b4
1e9919a0e3cf
83af27c511da
715a23f8fa61
022453f6b54f
0d4dd628d151
aa09037ef927
f9b02bed92ab
b284e9691d98
60c3dbd8f38a
87bff749a731
ba88c8578060
b4eeb7acd532
eb30f7cd26d1
903a86d48294



'b2bdd17cf59a\n2e7643b01412\n9c124ff553a7\nbeb3ff900509\n5e46e48fa372\n62700ada4e14\n5fa3c994bf84\n895703dde361\n4e945c1223c5\n50f42244bf01\n57f0c9ce29b4\n0b82a523a46c\n0f8a75b14cb0\n5f4d7300a2e7\ne1eb3ac50c2a\nd907274d448e\nd8483e94e35b\n45845e6c5afb\n171bc103e5b4\n1e9919a0e3cf\n83af27c511da\n715a23f8fa61\n022453f6b54f\n0d4dd628d151\naa09037ef927\nf9b02bed92ab\nb284e9691d98\n60c3dbd8f38a\n87bff749a731\nba88c8578060\nb4eeb7acd532\neb30f7cd26d1\n903a86d48294\n'

In [6]:
# Running 4k Random Write on 32 VFs

for i in range(1,33):
#     print("docker run -d -v /mnt/disk" + str(i) + ":/test -v /root/logs:/mylogs:rw fio_filesystem_randwrite/nvme:4k bash run_write.sh FioLog_nvme" + str(i) + ".json")
    connection.sendCommand("docker run -d -v /mnt/disk" + str(i) + ":/test -v /root/logs:/mylogs:rw fio_filesystem_randwrite/nvme:4k bash run_write.sh FioLog_nvme" + str(i) + ".json")
    print("Started FIO_RandRead_Container on NVMe disk" + str(i))
    

4b33459e9efb6edd106179f7ac71bb328a8d15bac341b2f08218cfbba1dd1475

Started FIO_RandRead_Container on NVMe disk1
1cbcc6cc7bf13f83771bdc2505657a8dc21a6f865c6e7cb1ddef738686f993e6

Started FIO_RandRead_Container on NVMe disk2
ebaede282990892d7f0033161a545d4855a243da1c7dbeea56df1af105fcbbdb

Started FIO_RandRead_Container on NVMe disk3
fcea131afaea0a283cb802711c863b7bd753c0f0b20efb5eb4c646c3064bb523

Started FIO_RandRead_Container on NVMe disk4
a7f91cdd3d5fd716e1e92235e4c9f2799f6c7277d56324173cb4abcadb221c08

Started FIO_RandRead_Container on NVMe disk5
1e468227b8d171d240af2a2364f459f84e13fd48e1292e9f20818dea1d1a77c8

Started FIO_RandRead_Container on NVMe disk6
9a013b3c01df94d1b7b9021f7eac3665df93a7e63868d2926ab14d0c8198454b

Started FIO_RandRead_Container on NVMe disk7
161ae9fdf6113c6c6d8d96d4feb19e3fcc538651dfdf2d44de24cae2fb018762

Started FIO_RandRead_Container on NVMe disk8
ea3244f77de1759ed75541d559c6b2be38da302e1328884c7f1e5fac805846c1

Started FIO_RandRead_Container on NVMe disk9
6

In [9]:
# Running 4k Random Reads on 32 VFs

for i in range(1,33):
#     print("docker run -d -v /mnt/disk" + str(i) + ":/test -v /root/logs:/mylogs:rw fio_filesystem_randread/nvme:4k bash run_read.sh FioLog_nvme" + str(i) + ".json")
    connection.sendCommand("docker run -d -v /mnt/disk" + str(i) + ":/test -v /root/logs:/mylogs:rw fio_filesystem_randread/nvme:4k bash run_read.sh FioLog_nvme" + str(i) + ".json")
    print("Started FIO_RandRead_Container on NVMe disk" + str(i))
    

eb30f7cd26d17b22b162eec7de6fb328e77a288c0c702a366da84cad824bedec

Started FIO_RandRead_Container on NVMe disk1
b4eeb7acd53207fc0692f5631ecd0ec91e1e9f02237700fbcb401a4dc9b4f8ae

Started FIO_RandRead_Container on NVMe disk2
ba88c85780605733e3ee013580828b47984b0c920c7d7e82e433494fb8907221

Started FIO_RandRead_Container on NVMe disk3
87bff749a731416814f755c4871bd0a5963572f3e5077dfc4c766174279c3f99

Started FIO_RandRead_Container on NVMe disk4
60c3dbd8f38aec6234860e4aadad4cc211bd47f47631ef65606e844e032e5131

Started FIO_RandRead_Container on NVMe disk5
b284e9691d987dc274b06a4e47dad6bcdc398987b055b43bbf03231634a02eb9

Started FIO_RandRead_Container on NVMe disk6
f9b02bed92ab41420bf391e3b77c6997056a648aeaadb2b84fd452fb4b57cdd3

Started FIO_RandRead_Container on NVMe disk7
aa09037ef927be9bdd8551997a565a9542ded6686338c90ad7acd66af71154ab

Started FIO_RandRead_Container on NVMe disk8
0d4dd628d15183da2e20457f28aebcb186413a742e818be3f5595f873fe0f132

Started FIO_RandRead_Container on NVMe disk9
0

In [27]:
# connection.sendCommand("docker run -d -v /mnt/disk1:/test -v /root/logs:/mylogs:rw docker_test/nvme:4k bash run.sh code.json")

import configparser
config = configparser.ConfigParser(allow_no_value=True,)

for device in range(1,4):



    config['global'] = {'ioengine': 'libaio',
                         'iodepth': 32,
                         'direct': 1,
                        'invalidate':1,
                        'runtime':600,
                        'time_based':1}

    config['global']['filename'] = '/dev/'+'nvme' + str(0) +'n' + str(device)

    config['rand-read-4k'] = {'bs': '128k',
                         'rw': 'read',
                        'stonewall':1}



    with open('write'+'_nvme_'+str(device)+'.fio', 'w') as configfile:
        config.write(configfile)
        
        
# for device in range(1,33):
#     cmd = connection.sendCommand('docker run -d -v /mnt/disk' + str(device) + ':/test -v /root/logs:/mylogs:rw docker_test/nvme:4k bash run.sh'+' '+'write_nvme_'+str(device)+'.fio'+ ' ' +'output_'+str(device)+'.json)


In [20]:


def ssh_edit_file(file_in, file_out, key, new):
    """
    SSH_EDIT_FILE - replace a line in a remote file
    """ 

    import warnings

    try:
#         s = new.replace("'","\'")
        if file_in == file_out:
            print('cd logs/fio/;sed -i "s/' + key + '/' + new + '/g" ' + file_in)
            connection.sendCommand('cd logs/fio/;sed -i "s/' + key + '/' + new + '/g" ' + file_in)

        else:
            connection.sendCommand('cd logs/fio/;sed -i "s/' + key + '/' + new + '/g" ' + 
                    file_in + '>|' +  file_out)
    except:
        warnings.warn('key not found')

ssh_edit_file("eval_write_seq_raw_128k.fio", "eval_write_seq_raw_128k.fio", "nvme0n2", "nvme0n1")
docker run -d --device /dev/nvme0n1 -v /root/logs:/mylogs:rw fio_raw_seq/nvme:128k bash run_raw_write.sh eval_write_seq_raw_128k.fio raw_nvme1.json

cd logs/fio/;sed -i "s/nvme0n2/nvme0n1/g" eval_write_seq_raw_128k.fio



In [24]:
# Running 4k Seq Writes on 32 VFs raw device

for i in range(1,33):
#     print("docker run -d -v /mnt/disk" + str(i) + ":/test -v /root/logs:/mylogs:rw fio_filesystem_randread/nvme:4k bash run_read.sh FioLog_nvme" + str(i) + ".json")
    connection.sendCommand("docker run -d --device /dev/nvme0n" + str(i) + " -v /root/logs:/mylogs:rw fio_raw_seq/nvme:128k bash run_raw_write.sh eval_write_seq_raw_128k.fio raw_nvme" + str(i) + ".json")
    #print("docker run -d --device /dev/nvme0n" + str(i) + " -v /root/logs:/mylogs:rw fio_raw_seq/nvme:128k bash run_raw_write.sh eval_write_seq_raw_128k.fio raw_nvme" + str(i) + ".json")
    ssh_edit_file("eval_write_seq_raw_128k.fio", "eval_write_seq_raw_128k.fio", "nvme0n" + str(i), "nvme0n" + str(i+1))
    #print("eval_write_seq_raw_128k.fio", "eval_write_seq_raw_128k.fio", "nvme0n" + str(i), "nvme0n" + str(i+1))
    print("Started FIO_RandRead_Container on NVMe disk" + str(i))

23b72a38a5723ed33b1b119dc371cf9c14ba0033e9361e1cd61352b57e5b2b1f

cd logs/fio/;sed -i "s/nvme0n1/nvme0n2/g" eval_write_seq_raw_128k.fio

Started FIO_RandRead_Container on NVMe disk1
25c60fbbc0e5ceee301212292b34b51f946e5f25444f76d5f2a0d3710398e2db

cd logs/fio/;sed -i "s/nvme0n2/nvme0n3/g" eval_write_seq_raw_128k.fio

Started FIO_RandRead_Container on NVMe disk2
85049eb9a361dfc79455bac3c7f9d54faa83ad8740b07f8acfd83ea1fbe3a62f

cd logs/fio/;sed -i "s/nvme0n3/nvme0n4/g" eval_write_seq_raw_128k.fio

Started FIO_RandRead_Container on NVMe disk3
334cb61b08e261f978d8e7c6c685de2137c8979cac6386951759a50d6e00d112

cd logs/fio/;sed -i "s/nvme0n4/nvme0n5/g" eval_write_seq_raw_128k.fio

Started FIO_RandRead_Container on NVMe disk4
45509ac6c7dc09f66cea18d86544bc7d3f94e1595e3bea06196f617c06760db4

cd logs/fio/;sed -i "s/nvme0n5/nvme0n6/g" eval_write_seq_raw_128k.fio

Started FIO_RandRead_Container on NVMe disk5
f14f8794dd88614db62f630b9ab620b804a62d6f5ad6f929bb8a31105ae5ed9c

cd logs/fio/;sed -i "s/n

In [25]:


# client = docker.from_env()
# print(client)
# client.containers.list()
#     print(container)
#     container.stop()

# WAIT_SECONDS = 10

def run_fio():
    output = connection.sendCommand("docker ps -a")
    docker = bool(re.search("[Uu]p", str(output)))
    print(docker)
#     docker = docker.group()
    if docker == True:
        print("FIO Containers are running, print status every 60 sec")
    else:
        print("FIO Containers exited , Test Complete")

    time.sleep(60)
    return(docker)

while True:
    if run_fio() == False:
        break

CONTAINER ID        IMAGE                   COMMAND                  CREATED              STATUS              PORTS               NAMES
52e128df3272        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   About a minute ago   Up About a minute                       sleepy_babbage
d8d484fd662d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   About a minute ago   Up About a minute                       frosty_yonath
ab421af57a0d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   About a minute ago   Up About a minute                       cocky_torvalds
3cda272ec7fc        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   About a minute ago   Up About a minute                       laughing_villani
ddd18150934f        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   About a minute ago   Up About a minute                       exciting_wilson
a497d7318f53        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   About a minute ago   Up About a minute                       crazy_g

CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS               NAMES
52e128df3272        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   3 minutes ago       Up 3 minutes                            sleepy_babbage
d8d484fd662d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   3 minutes ago       Up 3 minutes                            frosty_yonath
ab421af57a0d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   3 minutes ago       Up 3 minutes                            cocky_torvalds
3cda272ec7fc        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   3 minutes ago       Up 3 minutes                            laughing_villani
ddd18150934f        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   3 minutes ago       Up 3 minutes                            exciting_wilson
a497d7318f53        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   3 minutes ago       Up 3 minutes                            crazy_goldberg

CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS               NAMES
52e128df3272        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   5 minutes ago       Up 5 minutes                            sleepy_babbage
d8d484fd662d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   5 minutes ago       Up 5 minutes                            frosty_yonath
ab421af57a0d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   5 minutes ago       Up 5 minutes                            cocky_torvalds
3cda272ec7fc        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   5 minutes ago       Up 5 minutes                            laughing_villani
ddd18150934f        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   5 minutes ago       Up 5 minutes                            exciting_wilson
a497d7318f53        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   5 minutes ago       Up 5 minutes                            crazy_goldberg

CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS               NAMES
52e128df3272        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   7 minutes ago       Up 7 minutes                            sleepy_babbage
d8d484fd662d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   7 minutes ago       Up 7 minutes                            frosty_yonath
ab421af57a0d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   7 minutes ago       Up 7 minutes                            cocky_torvalds
3cda272ec7fc        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   7 minutes ago       Up 7 minutes                            laughing_villani
ddd18150934f        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   7 minutes ago       Up 7 minutes                            exciting_wilson
a497d7318f53        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   7 minutes ago       Up 7 minutes                            crazy_goldberg

CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS               NAMES
52e128df3272        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   9 minutes ago       Up 9 minutes                            sleepy_babbage
d8d484fd662d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   9 minutes ago       Up 9 minutes                            frosty_yonath
ab421af57a0d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   9 minutes ago       Up 9 minutes                            cocky_torvalds
3cda272ec7fc        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   9 minutes ago       Up 9 minutes                            laughing_villani
ddd18150934f        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   9 minutes ago       Up 9 minutes                            exciting_wilson
a497d7318f53        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   9 minutes ago       Up 9 minutes                            crazy_goldberg

CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS               NAMES
52e128df3272        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   11 minutes ago      Up 11 minutes                           sleepy_babbage
d8d484fd662d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   11 minutes ago      Up 11 minutes                           frosty_yonath
ab421af57a0d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   11 minutes ago      Up 11 minutes                           cocky_torvalds
3cda272ec7fc        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   11 minutes ago      Up 11 minutes                           laughing_villani
ddd18150934f        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   11 minutes ago      Up 11 minutes                           exciting_wilson
a497d7318f53        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   11 minutes ago      Up 11 minutes                           crazy_goldberg

CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS               NAMES
52e128df3272        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   13 minutes ago      Up 13 minutes                           sleepy_babbage
d8d484fd662d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   13 minutes ago      Up 13 minutes                           frosty_yonath
ab421af57a0d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   13 minutes ago      Up 13 minutes                           cocky_torvalds
3cda272ec7fc        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   13 minutes ago      Up 13 minutes                           laughing_villani
ddd18150934f        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   13 minutes ago      Up 13 minutes                           exciting_wilson
a497d7318f53        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   13 minutes ago      Up 13 minutes                           crazy_goldberg

CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS               NAMES
52e128df3272        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   15 minutes ago      Up 15 minutes                           sleepy_babbage
d8d484fd662d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   15 minutes ago      Up 15 minutes                           frosty_yonath
ab421af57a0d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   15 minutes ago      Up 15 minutes                           cocky_torvalds
3cda272ec7fc        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   15 minutes ago      Up 15 minutes                           laughing_villani
ddd18150934f        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   15 minutes ago      Up 15 minutes                           exciting_wilson
a497d7318f53        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   15 minutes ago      Up 15 minutes                           crazy_goldberg

CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS               NAMES
52e128df3272        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   17 minutes ago      Up 17 minutes                           sleepy_babbage
d8d484fd662d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   17 minutes ago      Up 17 minutes                           frosty_yonath
ab421af57a0d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   17 minutes ago      Up 17 minutes                           cocky_torvalds
3cda272ec7fc        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   17 minutes ago      Up 17 minutes                           laughing_villani
ddd18150934f        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   17 minutes ago      Up 17 minutes                           exciting_wilson
a497d7318f53        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   17 minutes ago      Up 17 minutes                           crazy_goldberg

CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS               NAMES
52e128df3272        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   19 minutes ago      Up 19 minutes                           sleepy_babbage
d8d484fd662d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   19 minutes ago      Up 19 minutes                           frosty_yonath
ab421af57a0d        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   19 minutes ago      Up 19 minutes                           cocky_torvalds
3cda272ec7fc        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   19 minutes ago      Up 19 minutes                           laughing_villani
ddd18150934f        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   19 minutes ago      Up 19 minutes                           exciting_wilson
a497d7318f53        fio_raw_seq/nvme:128k   "bash run_raw_write.…"   19 minutes ago      Up 19 minutes                           crazy_goldberg

In [ ]:
mkdir /mnt/disk1
mkfs.ext4 /dev/nvme0n2
mount /dev/nvme0n2 /mnt/disk1

docker run -it -v /mnt/disk1:/test nvme_vf/randread_30g_timebased:4k > raw


docker run -d -v /mnt/disk1:/test -v /root/logs:/mylogs:rw docker_test/nvme:4k bash run.sh karan.json
docker run -d --device /dev/nvme0n1 -v /root/logs:/mylogs:rw fio_raw_seq/nvme:128k bash run_raw_write.sh eval_write_seq_raw_128k.fio raw_nvme1.json